<a href="https://colab.research.google.com/github/EmiliaFidler/Intro_to_Comp_Ling_WS24/blob/main/final_project_comp_ling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Project**
---


*VU Introduction to Computational Linguistics - WS24/25*

---
**Topic**: Toxicity Detection in Social Media  
Detecting toxic or offensive language in social media, e.g. YouTube comments in Polish and English    

**Datasets**: PolishCyberbullyingDataset or hatecheck-polish for the Polish dataset and civil_comments or toxicity_classification_jigsaw for the English dataset
model: XLM-RoBERTa Base   



**Set-Up:**
Installing librarires required for the task

In [1]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install scikit-learn
!pip install matplotlib seaborn
!pip install optuna
!pip install tensorboard
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 25.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset, DatasetDict

# English dataset (civil_comments)
dataset_en = load_dataset("civil_comments")

# Polish dataset (PolishCyberbullyingDataset)
dataset_pl = load_dataset("ptaszynski/PolishCyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.73k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

v2_training_anonymized.tsv:   0%|          | 0.00/801k [00:00<?, ?B/s]

v2_test_anonymized.tsv:   0%|          | 0.00/79.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10038 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
print(dataset_en)
print(dataset_pl)

DatasetDict({
    train: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 1804874
    })
    validation: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
    test: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
})
DatasetDict({
    train: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 10038
    })
    test: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 1000
    })
})


In [4]:
train_valid_split = dataset_pl['train'].train_test_split(test_size=0.1, seed=42)

In [5]:
train_en = dataset_en['train']
val_en = dataset_en['validation']
test_en = dataset_en['test']

train_pl = train_valid_split['train']
val_pl = train_valid_split['test']
test_pl = dataset_pl['test']

In [6]:
print("Train size:", len(train_pl))
print("Validation size:", len(val_pl))
print("Test size:", len(test_pl))

print("Train size:", len(train_en))
print("Validation size:", len(val_en))
print("Test size:", len(test_en))

Train size: 9034
Validation size: 1004
Test size: 1000
Train size: 1804874
Validation size: 97320
Test size: 97320


Preprocessing!

In [7]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
  input_column = 'text' if 'text' in examples else 'TEXT'
  return tokenizer(examples[input_column], truncation=True, padding="max_length", max_length=128, return_tensors="pt")

# tokenizing dataset
tokenized_train_en = train_en.map(tokenize_function, batched=True)
tokenized_val_en = val_en.map(tokenize_function, batched=True)
tokenized_test_en = test_en.map(tokenize_function, batched=True)

tokenized_train_pl = train_pl.map(tokenize_function, batched=True)
tokenized_val_pl = val_pl.map(tokenize_function, batched=True)
tokenized_test_pl = test_pl.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # for dynamic padding during training

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/9034 [00:00<?, ? examples/s]

Map:   0%|          | 0/1004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
# Define save paths in your Drive
save_path = '/content/drive/MyDrive/tokenized_datasets/'

# Save English datasets
tokenized_train_en.save_to_disk(save_path + 'tokenized_train_en')
tokenized_val_en.save_to_disk(save_path + 'tokenized_val_en')
tokenized_test_en.save_to_disk(save_path + 'tokenized_test_en')

# Save Polish datasets
tokenized_train_pl.save_to_disk(save_path + 'tokenized_train_pl')
tokenized_val_pl.save_to_disk(save_path + 'tokenized_val_pl')
tokenized_test_pl.save_to_disk(save_path + 'tokenized_test_pl')

Saving the dataset (0/4 shards):   0%|          | 0/1804874 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/97320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/97320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9034 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1004 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]